# Ensemble Method

Technique that creates multiple models and combines then to produce better results.

## Random Forest

Ensemble learning method that constructs collection of decision trees and aggrgates the predictions of each tree.

- Versatile
- Can be used for classification or regression (categorical, continous)
- Accepts various types of inputs
- Easily handles outliers
- Generates feature importance score
- Less likely to overfit

## Read in text, clean and vectorize 

In [5]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
x_tfidf = tfidf_vect.fit_transform(data['body_text'])

x_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(x_tfidf.toarray())], axis = 1)
x_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
from sklearn.ensemble import RandomForestClassifier

In [8]:
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_estimator_type', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_required_parameters', '_set_oob_score', '_validate_X_predict', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_sample

In [16]:
from sklearn.model_selection import KFold, cross_val_score

#K learn splits data into fold and cross val gets the score for each fold

In [17]:
rf = RandomForestClassifier(n_jobs = -1)
k_fold = KFold(n_splits = 5)
cross_val_score(rf, x_features, data['label'], cv=k_fold, scoring='accuracy', n_jobs=-1)

array([0.97307002, 0.97845601, 0.97304582, 0.96675651, 0.96855346])

# Using Holdout Set

In [19]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x_features, data['label'], test_size = 0.2)

In [28]:
rf2 = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf2_model = rf2.fit(x_train, y_train)

In [31]:
sorted(zip(rf2_model.feature_importances_, x_train.columns), reverse=True) [0:10]

[(0.06535963327342263, 'body_len'),
 (0.050293099698135835, 2031),
 (0.035511557894766545, 4796),
 (0.035165008534836735, 7350),
 (0.03140962727615069, 5724),
 (0.030426381013440515, 3134),
 (0.029068906882621315, 1803),
 (0.021549554005276382, 7027),
 (0.019935625587994943, 295),
 (0.019007127340767774, 6746)]

In [34]:
y_pred = rf_model.predict(x_test)
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

In [41]:
print('Precision: {} / Recall: {} / Accuracy : {}'.format(round(precision,3),
                                                         round(recall, 3),
                                                         round((y_pred==y_test).sum()/len(y_pred),3) 
                                                         ))

Precision: 1.0 / Recall: 0.599 / Accuracy : 0.943
